In [1]:
#many parameters and mappings from yesterday hardcoded so as not to run them again as quite time consuming
#today was spent looking through many other classifiers and trying to combine classifiers
#considering the time spent there was very little success with only a marginal gain from a very different
#random forest then combingin it with my random forest from yesterday with a voting classifier
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('C:/Users/danie/Desktop/train_features.csv')

In [3]:
region_dict={'Arusha': 2, 'Dar es Salaam': 7, 'Dodoma': 1, 'Iringa': 11, 'Kagera': 18, 'Kigoma': 16, 'Kilimanjaro': 3,
             'Lindi': 80, 'Manyara': 21, 'Mara': 20, 'Mbeya': 12, 'Morogoro': 5, 'Mtwara': 90, 'Mwanza': 19, 'Pwani': 6, 
             'Rukwa': 15, 'Ruvuma': 10, 'Shinyanga': 17, 'Singida': 13, 'Tabora': 14, 'Tanga': 4}
def clean_region(frame):
    frame['region_code']=frame['region'].map(region_dict)

In [4]:
clean_region(df)

In [5]:
#this bit of code will clean up all the non entries in long and lat using a mean for region/area if possible, if not just region
df['my_area_code']=100*df['region_code']+df['district_code']
df_long_lat=df[['region_code','my_area_code','longitude','latitude']].copy()
df_long_lat=df_long_lat[df_long_lat['longitude']!=0]
df_ll=df_long_lat.groupby(['my_area_code'])[['longitude','latitude']].mean()
longitude_dict=df_ll['longitude'].to_dict()
latitude_dict=df_ll['latitude'].to_dict()
df_ll2=df_long_lat.groupby(['region_code'])[['longitude','latitude']].mean()
longitude_dict2=df_ll2['longitude'].to_dict()
latitude_dict2=df_ll2['latitude'].to_dict()


def clean_long_lat(frame):
    frame['my_area_code']=100*frame['region_code']+frame['district_code']
    frame['latitude_est']=frame['my_area_code'].map(latitude_dict)
    frame['longitude_est']=frame['my_area_code'].map(longitude_dict)
    frame['latitude_est2']=frame['region_code'].map(latitude_dict2)
    frame['longitude_est2']=frame['region_code'].map(longitude_dict2)
    frame['new_longitude']=np.where(frame['longitude']!=0,frame['longitude'],frame['longitude_est'])
    frame['new_longitude']=np.where(frame['new_longitude'].isna(),frame['longitude_est2'],frame['new_longitude'])
    frame['new_latitude']=np.where(frame['latitude']!=0,frame['latitude'],frame['latitude_est'])
    frame['new_latitude']=np.where(frame['new_latitude'].isna(),frame['latitude_est2'],frame['new_latitude'])
    frame=frame.drop(['my_area_code','latitude_est','longitude_est','latitude_est2','longitude_est2'],axis=1)

In [6]:
clean_long_lat(df)

In [7]:
features=['basin','region','public_meeting','scheme_management','permit','extraction_type',
 'extraction_type_group','extraction_type_class','management','management_group','payment',
 'payment_type','water_quality','quality_group','quantity','quantity_group','source','source_type',
 'source_class','waterpoint_type','waterpoint_type_group','amount_tsh','gps_height','new_longitude','new_latitude',
 'region_code','district_code','population','construction_year']

In [8]:
X=df.copy()[features]

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
preprocessor = make_pipeline(ce.OrdinalEncoder(),SimpleImputer(),RobustScaler())

preprocessor.fit(X)

def process_data(frame):
    X=frame.copy()[features]
    X = preprocessor.transform(X)
    X=pd.DataFrame(X,columns=features)

    return X

In [9]:
X=process_data(X)

In [10]:
y=pd.read_csv('C:/Users/danie/Desktop/train_labels.csv')
y=y.drop('id',axis=1)
y_dict={'functional':1,'non functional':0,'functional needs repair':2}
y['status_group']=y['status_group'].map(y_dict)
y.head()

,status_group
0,1
1,1
2,1
3,0
4,1


In [22]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [23]:
model=RandomForestClassifier(max_depth=20,n_estimators=133,n_jobs=-1,random_state=42)
model.fit(X,y)

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=133, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [24]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(model, X, y, cv=5)
cv_results

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y wa

{'fit_time': array([12.58218074,  9.06301546,  9.99330711, 10.36168003, 10.77766037]),
 'score_time': array([0.34390068, 0.35938978, 0.46891046, 0.45314121, 0.46892214]),
 'test_score': array([0.81348371, 0.80877031, 0.81043771, 0.80816498, 0.80771174]),
 'train_score': array([0.94993582, 0.95105116, 0.95065236, 0.95      , 0.95185388])}

In [22]:
#ada boost is worse than just normal!!!  seems to be way over fitting with 50 estimators socres were 99.5/79
#tryin 5 estimators next
from sklearn.ensemble import AdaBoostClassifier
ada_model=AdaBoostClassifier(base_estimator=model,n_estimators=2,random_state=42)
cv_results_ada = cross_validate(ada_model, X, y, cv=3)
cv_results_ada

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If y

{'fit_time': array([17.20932078, 21.22799182, 20.59823942]),
 'score_time': array([1.59440327, 1.37335134, 1.42033839]),
 'test_score': array([0.805     , 0.80656566, 0.80439394]),
 'train_score': array([0.9735101 , 0.97396465, 0.97421717])}

In [18]:
from sklearn.ensemble import ExtraTreesClassifier
et_model=ExtraTreesClassifier(n_estimators=133,max_depth=20,random_state=42)
cv_results_et = cross_validate(et_model, X, y, cv=3)
cv_results_et

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('t

{'fit_time': array([12.26110387, 12.35414386, 13.35739875]),
 'score_time': array([1.44036627, 1.45536876, 1.52439427]),
 'test_score': array([0.79505051, 0.79727273, 0.79459596]),
 'train_score': array([0.92790404, 0.92838384, 0.92979798])}

In [19]:
from sklearn.model_selection import RandomizedSearchCV
param_distributions_et = {
    'n_estimators':randint(75,150),
    'max_depth':randint(10,22)
}

search_et=RandomizedSearchCV(
      estimator=ExtraTreesClassifier(n_jobs=-1,random_state=42),
      param_distributions=param_distributions_et,
      n_iter=30,
      scoring='accuracy',
      n_jobs=-1,
      cv=3,
      verbose=10,
      return_train_score=True)

search_et.fit(X,y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  8.3min finished
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000289607ECB70>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000289607ECCF8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [20]:
pd.DataFrame(search_et.cv_results_).sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
7,16.675883,0.394455,2.599996,0.140616,21,105,"{'max_depth': 21, 'n_estimators': 105}",0.796515,0.798939,0.796465,0.797306,0.001155,1,0.940379,0.941515,0.942778,0.941557,0.000980
19,15.070471,0.383526,2.629664,0.136685,21,94,"{'max_depth': 21, 'n_estimators': 94}",0.796667,0.798990,0.795808,0.797155,0.001344,2,0.940253,0.941591,0.943056,0.941633,0.001145
4,16.315252,0.318128,2.551317,0.391925,19,109,"{'max_depth': 19, 'n_estimators': 109}",0.794091,0.798081,0.794242,0.795471,0.001846,3,0.912197,0.912879,0.915227,0.913434,0.001298
24,19.145507,0.144305,3.280847,0.725761,20,111,"{'max_depth': 20, 'n_estimators': 111}",0.794192,0.796919,0.794747,0.795286,0.001177,4,0.928763,0.928687,0.930328,0.929259,0.000757
11,15.985056,0.113623,2.356929,0.099962,20,100,"{'max_depth': 20, 'n_estimators': 100}",0.794192,0.796616,0.794293,0.795034,0.001120,5,0.927778,0.928535,0.930177,0.928830,0.001001


In [28]:
from sklearn.ensemble import BaggingClassifier
bag_model=BaggingClassifier(base_estimator=model,n_estimators=5,random_state=42)
cv_results_bag = cross_validate(bag_model, X, y, cv=5)
cv_results_bag

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

{'fit_time': array([31.64811659, 37.66581678, 37.23910594, 35.51166868, 35.36774278]),
 'score_time': array([2.44427061, 2.8417089 , 2.20325089, 2.28152752, 2.40769744]),
 'test_score': array([0.81213703, 0.80725528, 0.81010101, 0.80749158, 0.80653309]),
 'train_score': array([0.91045687, 0.91228772, 0.91165825, 0.91188973, 0.9146711 ])}

In [29]:
cv_results_bag['test_score'].mean()

0.8087035972031265

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
model_gbc=GradientBoostingClassifier(max_depth=6,n_estimators=147,learning_rate=0.175,random_state=42)
model_gbc.fit(X,y)

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.175, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=147,
              n_iter_no_change=None, presort='auto', random_state=42,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [26]:
from sklearn.ensemble import VotingClassifier
model_et=search_et.best_estimator_
vc1 = VotingClassifier(estimators=[('rf', model), ('et', model_et),('gbc', model_gbc)], voting='soft',weights=[2,1,1])

In [27]:
cv_results_vc = cross_validate(vc1, X, y, cv=3)
cv_results_vc

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

{'fit_time': array([199.71453476, 198.47064114, 197.0310421 ]),
 'score_time': array([2.14531398, 1.96343255, 2.09928727]),
 'test_score': array([0.80530303, 0.80757576, 0.80257576]),
 'train_score': array([0.93984848, 0.94159091, 0.94204545])}

In [33]:
from sklearn.ensemble import VotingClassifier
model_rf2=search_rf.best_estimator_
vc2=VotingClassifier(estimators=[('rf1', model), ('rf2', model_rf2)], voting='soft',weights=[1,1])
cv_results_vc2 = cross_validate(vc2, X, y, cv=5)
cv_results_vc2

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

{'fit_time': array([20.89116287, 22.09659958, 24.35471725, 27.42200637, 26.15644979]),
 'score_time': array([0.89458656, 0.95331717, 0.93758845, 0.88291621, 1.47725701]),
 'test_score': array([0.81516707, 0.80809696, 0.81102694, 0.80858586, 0.80981647]),
 'train_score': array([0.9523559 , 0.95300827, 0.95324074, 0.95208333, 0.95435798])}

In [34]:
cv_results_vc2['test_score'].mean()

0.8105386594090879

In [44]:
vc2.fit(X,y)

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingClassifier(estimators=[('rf1', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_wei...imators=141, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft',
         weights=[1, 1])

In [36]:
test=pd.read_csv('C:/Users/danie/Desktop/test_features.csv')
clean_region(test)
clean_long_lat(test)
X_test=test[features].copy()
X_test=process_data(X_test)
X_test.head()

,basin,region,public_meeting,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,...,waterpoint_type,waterpoint_type_group,amount_tsh,gps_height,new_longitude,new_latitude,region_code,district_code,population,construction_year
0,0.0,-0.727273,0.0,2.25,0.0,1.00,1.333333,1.0,8.0,3.0,...,1.0,2.0,0.0,1.233276,0.097800,0.184468,0.750000,0.000000,1.376744,0.012974
1,-0.4,0.545455,0.0,0.00,0.0,-0.25,-0.333333,-0.5,0.0,0.0,...,-0.5,0.0,0.0,0.909608,0.447449,0.328391,-0.833333,-0.333333,1.279070,0.006986
2,0.0,0.727273,0.0,0.00,1.0,1.00,1.333333,1.0,0.0,0.0,...,1.0,2.0,0.0,0.908092,-0.036063,0.003309,0.083333,-0.333333,2.209302,0.011976
3,-0.2,0.363636,1.0,0.00,0.0,1.00,1.333333,1.0,0.0,0.0,...,1.0,2.0,0.0,-0.077317,0.806168,-0.843246,5.666667,13.333333,1.046512,0.000499
4,-0.2,-0.090909,1.0,1.25,0.0,-0.25,-0.333333,-0.5,4.0,0.0,...,-0.5,0.0,25.0,0.675384,0.024928,-1.136994,-0.166667,0.000000,0.162791,0.006986


In [37]:
reverse_dict={v: k for k, v in y_dict.items()}
def subber(frame,y_pred):
    temp_frame=pd.DataFrame(frame.copy()['id'])
    temp_frame['status_group']=y_pred
    temp_frame['status_group']=temp_frame['status_group'].map(reverse_dict)

    return temp_frame

In [45]:
y_pred_new=vc2.predict(X_test)
sub_df4 = subber(test,y_pred_new)
sub_df4.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [46]:
sub_df4.to_csv('C:/Users/danie/Desktop/sub_csv4',index=False)


In [28]:
bag_model2=BaggingClassifier(base_estimator=vc1,n_estimators=5,random_state=42)
cv_results_bag2 = cross_validate(bag_model2, X, y, cv=3)
cv_results_bag2

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:622: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If y

{'fit_time': array([869.23912406, 839.24491215, 836.7302084 ]),
 'score_time': array([10.56511188, 10.79598475, 10.80519962]),
 'test_score': array([0.80409091, 0.80626263, 0.80191919]),
 'train_score': array([0.90717172, 0.90762626, 0.9089899 ])}

In [30]:
param_distributions_rf = {
    #'criterion':['gini','entropy'],
  #  'min_samples_split':randint(2,5),
   # 'min_samples_leaf':randint(1,4),
    'max_depth':randint(17,21),
 #   'max_features':randint(25,30),
    'n_estimators':randint(130,145)
}

search_rf=RandomizedSearchCV(
      estimator=RandomForestClassifier(criterion='entropy',n_jobs=-1,random_state=42),   #max_depth=20
      param_distributions=param_distributions_rf,
      n_iter=10,
      scoring='accuracy',
      n_jobs=-1,
      cv=5,
      verbose=10,
      return_train_score=True)

search_rf.fit(X,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 11.6min finished
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000227081740F0>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000022708174DD8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='accuracy', verbose=10)

In [31]:
#best params for entropy are depth 20, estimators 141
pd.DataFrame(search_rf.cv_results_).sort_values(by='rank_test_score').head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,51.143096,0.286689,1.696038,0.272270,20,141,"{'max_depth': 20, 'n_estimators': 141}",0.815840,0.807592,0.809428,...,0.809899,0.003064,1,0.954376,0.954544,0.954440,0.953725,0.955705,0.954558,0.000641
8,48.451389,0.286307,2.299252,0.521919,20,143,"{'max_depth': 20, 'n_estimators': 143}",0.815840,0.806666,0.809343,...,0.809613,0.003228,2,0.954271,0.954776,0.954840,0.953809,0.955642,0.954667,0.000614
0,47.567206,1.605692,2.000313,0.214017,20,144,"{'max_depth': 20, 'n_estimators': 144}",0.816009,0.806919,0.809259,...,0.809579,0.003301,3,0.954292,0.954734,0.954777,0.953830,0.955852,0.954697,0.000672
9,38.566156,10.484699,1.564799,0.909835,19,136,"{'max_depth': 19, 'n_estimators': 136}",0.812810,0.806834,0.810690,...,0.809057,0.002312,4,0.938467,0.939835,0.941162,0.939962,0.942153,0.940316,0.001254
6,44.579488,0.454898,2.033656,0.390706,18,137,"{'max_depth': 18, 'n_estimators': 137}",0.813989,0.805824,0.809848,...,0.808906,0.002849,5,0.924157,0.923567,0.925779,0.925316,0.926645,0.925093,0.001107


In [29]:
len(X.columns)

29

In [21]:
X.head()

,basin,region,public_meeting,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,...,waterpoint_type,waterpoint_type_group,amount_tsh,gps_height,new_longitude,new_latitude,region_code,district_code,population,construction_year
0,-0.8,-0.909091,0.0,0.00,-1.0,-0.25,-0.333333,-0.5,0.0,0.0,...,-0.5,0.0,300.00,0.773925,0.007513,-0.927176,-0.083333,0.666667,0.390698,0.006487
1,-0.6,-0.818182,1.0,0.25,0.0,-0.25,-0.333333,-0.5,1.0,0.0,...,-0.5,0.0,0.00,0.780747,-0.053751,0.551184,0.666667,-0.333333,1.186047,0.011976
2,-0.4,-0.727273,0.0,0.00,0.0,-0.25,-0.333333,-0.5,0.0,0.0,...,0.0,0.0,1.25,0.240288,0.653248,0.230180,0.750000,0.333333,1.046512,0.011477
3,-0.2,-0.636364,0.0,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.00,-0.080349,0.915758,-1.176286,6.500000,20.000000,0.153488,0.000000
4,-0.6,-0.545455,0.0,0.50,0.0,-0.25,-0.333333,-0.5,2.0,1.0,...,-0.5,0.0,0.00,-0.279704,-0.967077,0.612956,0.500000,-0.666667,-0.116279,-0.991018


In [25]:
alt_rf=search_rf.best_estimator_
cv_results_rf = cross_validate(alt_rf, X, y, cv=5)
cv_results_rf

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: DataConversionWarning: A column-vector y wa

{'fit_time': array([41.91098833, 48.93994403, 54.43369699, 58.0788362 , 48.62481451]),
 'score_time': array([0.34494591, 0.47239614, 0.5854001 , 0.34379268, 0.34388304]),
 'test_score': array([0.81070617, 0.80675027, 0.80833333, 0.80563973, 0.80628052]),
 'train_score': array([0.93680423, 0.94019234, 0.94339226, 0.93808923, 0.94183746])}

In [27]:
cv_results_rf['test_score'].mean()

0.8075420051278478